Clasificador y regresor lineal mediante SKlearn

In [0]:
import pandas as pd
import altair as alt
from google.colab import files
import numpy as np


from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize

#Se prepara todo lo necesario para hacer el clasificador
from sklearn import neighbors
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

1- Análisis de datos

In [0]:
juegos=pd.read_csv("games.csv",encoding='ISO-8859-1')
print(juegos.columns)


In [0]:
#Solo importa editora del juego y ventas en el resto del mundo, por lo que se eliminan los datos que no estén relacionados.
del juegos['Id']
del juegos['Name']
del juegos['Rank']
del juegos['Difficulty']
del juegos['Year']
juegos.columns

In [0]:
#Primero se verá en qué columnas hay valores nulos

print(juegos['Publisher'].isnull().values.sum())
print(juegos['Platform'].isnull().values.sum())
print(juegos['Genre'].isnull().values.sum())
print(juegos['Ign_score'].isnull().values.sum())
print(juegos['NA_Sales'].isnull().values.sum())
print(juegos['EU_Sales'].isnull().values.sum())
print(juegos['JP_Sales'].isnull().values.sum())
print(juegos['Other_Sales'].isnull().values.sum())



In [0]:

#Se van a eliminar todas las filas que no contengan ningún dato sobre ventas, al ser consideradas no útiles.
#Además, se elimina la columna de Ign_score porque más del 60% de datos son nulos.
del juegos['Ign_score']
juegos.dropna(subset=['NA_Sales','EU_Sales','JP_Sales','Other_Sales'],how='all',inplace=True)
#Se revisa ahora qué datos nulos quedan
print('Nulos en Platform:',juegos['Platform'].isnull().values.sum())
print('Nulos en Publisher:',juegos['Publisher'].isnull().values.sum())
print('Nulos en NA_Sales:',juegos['NA_Sales'].isnull().values.sum())
print('Nulos en EU_Sales',juegos['EU_Sales'].isnull().values.sum())
print('Nulos en JP_Sales:',juegos['JP_Sales'].isnull().values.sum())
print('Nulos en Other_Sales',juegos['Other_Sales'].isnull().values.sum())



In [0]:
#Se eliminan las filas que sean nulos en Publisher, porque este es uno de los datos principales para poder entrenar al programa después, además de ser
#menos del 0.1%
juegos.dropna(subset=['Publisher'],inplace=True)
print(juegos['Publisher'].isnull().values.sum())

juegos.columns

1.1 - Codificar datos

In [0]:
print(juegos['Platform'].unique())
print(len(juegos['Publisher'].unique()))
juegos.groupby(["Publisher"]).count().median()

In [0]:
# se crea un vector de Other_Sales y de Publisher, junto a la matriz asociada
vectorventas = juegos['Other_Sales']
vectorpublisher = juegos['Publisher']

matriz = juegos.copy()

del matriz['Publisher']
del matriz['Other_Sales']

In [0]:


#Con onehotencoder se pasa la columna de platform y genre a vectores 
one = pd.get_dummies(matriz['Platform'], prefix='Platform')
matriz = pd.concat([matriz, one], axis=1, sort=False)
del matriz['Platform']

one = pd.get_dummies(matriz['Genre'], prefix='Genre')
matriz = pd.concat([matriz, one], axis=1, sort=False)
del matriz['Genre']



#Se normalizan las columnas
#Este código se sacó de https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame
from sklearn import preprocessing

x = matriz.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
matriz = pd.DataFrame(x_scaled)

#Con esto ya se tiene la matriz normalizada junto a los vectores

2.1 Clasificador

In [0]:
#Se eliminó la columna Rank y mejoró el acurracy

#Se va a cambiar la matriz de prueba, agregando las columna de Other_Sales para tener más información y esperar que 
#el programa mejore su precisión, resultado: Se mantuvo en 17%, se vuelve a eliminar.

#Se agrega la columna de genre, además de cambiar k=50 y x_test=0.2. Resultado: Aumenta a un 20% 
#Se agrega la columna year. resultado: Disminuye a un 18%, por lo que se vuelve a eliminar.

#Con la nueva matriz se prueba
#Se cambia knn weights a distance. Empeora a un 18% de acurracy.
#algorithm='ball_tree' resultado: mantiene 20% , algorithm='kd_tree' mantiene 20%
#distance + brute = 18% acurracy
# Probando con diversos valores de n, se llega a que n=50 mejora los resultados


#Se define el clasificador
knn = neighbors.KNeighborsClassifier(n_neighbors=50, weights='uniform', algorithm='auto')
 #Se separa la matriz y el vectorpublisher en set de entrenamiento y set de testeo
X_train, X_test, y_train, y_test = train_test_split(matriz, vectorpublisher, test_size=0.2, random_state=0)
#Se entrena el clasificador
knn.fit(X_train, y_train)
#Se revisa su comportamiento
predicted = knn.predict(X_test)
print( metrics.classification_report(y_test, predicted))

2.2 Regresor

In [0]:
#La base de este código fue sacado del ejemplo de la clase 13 hecha por el profesor Denis Parra.

#fit intercept= False. Resultado MSE se mantiene
#normalize= True. Resultado MSE se mantiene (como era de esperarse)
#n_jobes se prueba con muchos números. Resultado MSE se mantiene

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


#Para entrenar el regresor, se separa la matriz y la columna Other_Sales en set de entrenamiento y de testeo
X_train, X_test, y_train, y_test = train_test_split(matriz, vectorventas, test_size=0.3, random_state=0)

# Se crea el regresor
regr = linear_model.LinearRegression( fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)

# Se entrena  el regresor con los set de entrenamiento
regr.fit(X_train, y_train)

# Se hace la predicción del set de testeo
y_pred = regr.predict(X_test)

In [0]:
#Estas son las predicciones hechas.
y_pred

In [0]:
#Se revisa el MSE, es muy bajo, por lo que la predicción es buena.
print(mean_squared_error(y_test, y_pred))


3.1 Redución de dimensionalidad

In [0]:
#Se usa tsne para reducir la dimensión
from sklearn.manifold import TSNE
tsne = TSNE(perplexity=50, n_iter=1000, verbose=1)
intermediario = tsne.fit_transform(matriz)

3.2 Visualizar datos reducidos 

In [0]:
#Se crea un dataframe de pandas para utilizarlo en el grafico
tsneDataframe = pd.DataFrame(intermediario,columns=["tsne-1","tsne-2"])
#Se desactiva el limite de datos de altair
alt.data_transformers.disable_max_rows()
#Se agrega la columna de Publisher para identificarla con colores en el grafico
tsneDataframeAlt = tsneDataframe
tsneDataframeAlt["Publisher"] = vectorpublisher
alt.Chart(tsneDataframeAlt).mark_circle(size=60).encode(
    x="tsne-1",
    y="tsne-2",
    color="Publisher"
)

3.3 Predecir las clases usando clustering

In [0]:
#Se usa kmeans para clustering con centroides
from sklearn.cluster import  KMeans
print(vectorpublisher.nunique())
#Como hay 578 Publishers se usan 578 centros
kmeans = KMeans(n_clusters=578, verbose=0, random_state=0).fit(intermediario)
kmeans.labels_
print(kmeans.cluster_centers_)

In [0]:
#Se usa dbscan para clustering por densidad
from sklearn.cluster import DBSCAN
#Se usa eps=0.5 por dar un resultado cercano al nuemero real de Publisher, lo mismo para min_saples=5, aunque este afecta menos el resultado
dbscan = DBSCAN(eps=0.5, min_samples=5, n_jobs=4).fit(intermediario)
dbscan.labels_
len(set(dbscan.labels_))

3.4 Visualizar los cluster

In [0]:
#Se agrega el label creado por kmeans como columna Publisher
tsneDataframeAlt = tsneDataframe
tsneDataframeAlt["Publisher"] = kmeans.labels_
alt.Chart(tsneDataframeAlt).mark_circle(size=60).encode(
    x="tsne-1",
    y="tsne-2",
    color="Publisher:N"
)

In [0]:
#Se agrega el label creado por dbscan como columna Publisher
tsneDataframeAlt = tsneDataframe
tsneDataframeAlt["Publisher"] = dbscan.labels_
alt.Chart(tsneDataframeAlt).mark_circle(size=60).encode(
    x="tsne-1",
    y="tsne-2",
    color="Publisher:N"
)

4- Preguntas

1- Eliminaste columnas en el dataset que no aportaban información? De ser así, ¿cuáles fueron y por qué? ¿Cómo resolviste el tema de los valores nulos?

 Eliminamos las columnas de Id y Name, porque en este caso no nos iteresaba poder identificar cada juego individualmente. También eliminamos las columnas Year, Difficulty y Rank después de probar cada una de estas columnas en el clasificador y ver que el acurracy disminuía si se incluían. Por último, se eliminó la columna Ign_score porque más del 60% de sus datos eran nulos, por lo que estaba demasiado incompleta como para poder usarla. Para los valores nulos, se eliminaron todas las filas que no tenían ningún tipo de información sobre ventas y al final se eliminaron las filas que no tenían Publisher (siendo estas menos del 0.1%)




2- ¿Qué normalizaste, filas o columnas? ¿Por qué? ¿Para qué sirve normalizar los datos?

  Se normalizaron las columnas, porque los datos se tienen que normalizar con respecto a los de su misma clase y se normalizó porque así todos los datos podían tener la misma importancia. Normalizar los datos sirve para que no haya un tipo de dato que tenga mayor peso que otro. Por ejemplo, si en la regresión la matriz no se hubiera normalizado, las columnas con información de ventas (que tienen valores desde 0 hasta 41.49) tendrían mucho más peso que las columnas de Genre y Platform, al ser estas últimas con valores 0 y 1. Para que todas las categorías tengan la misma importancia, se hace que las todas columnas tengan valores entre 0 y 1.



3- ¿Por qué se separan los datos en set de entrenamiento y set de pruebas? ¿Qué proporción de los datos utilizaste para cada uno y por qué?

 Se separan porque la idea es que los programas puedan entrenarse con un set y después ponerlo a prueba con otro set que nunca ha visto, para que de verdad sea una predicción y no sólo repita información. En el clasificador al comienzo se usó un 70% de entrenamiento y 30% de testeo, sin embargo, al analizar los resultados se concluyó que se necesitaba de más datos de entrenamiento para poder mejorar el acurracy. Entonces se cambió a un 80% de entrenamiento y 20% de testeo, partición que es muy común. Para el regresor se hizo un 70% de entrenamiento y 30% de testeo, porque no había diferencia entre el MSE de la partición 70/30 y 80/20

4- ¿Qué hiperparámetros modificaste para probar tu clasificador? ¿Y el regresor? ¿Cuáles combinaciones de parámetros te dieron mejores resultados y por qué crees que es así?

 Se modificó el número de vecinos a usar (n), el peso (weight) y el algoritmo. Se vio que n=50, weight='uniform' y algorithm='auto' maximizaba el acurracy, creemos que es porque al ser tantas publishers se necesitaba una muestra relativamente grande de puntos, y como los puntos eran tan dispersos no importaban tanto sus distancias con respecto al punto que se quiere clasificar. Para el regresor se cambió el fit_intercept, nomalize y n_jobs. Siempre se mantuvo el MSE. Esto tiene sentido, porque los datos ya venían normalizados.

5- Para el clasificador, explica la diferencia entre las métricas del set de pruebas para cada clase, ¿Qué nos dice de la calidad del clasificador por cada clase?. Justifica.

 En el resultado del clasificador, se ve que muchos publishers nunca se predijeron. Analizando la predicción hecha con el set de prueba, se concluye que la calidad del clasificador es mala, tienendo un acurracy de solo 0.2. Esto pasa porque hay una gran cantidad de publishers, y la mayoría de estos tienen muy poca información disponible, lo que dificulta el poder tener buenos puntos de comparación.




6- Al usar t-SNE reducimos la dimensionalidad de los datos. Nombra dos razones para hacer eso.

 Una razón es que, al tener menos componentes por dato estos son mas fáciles de procesar y almacenar, ya que cualquier cálculo que se haga sobre los datos va a tener menos variables y se va a demorar menos. Otra razón es que al haber menos variables por dato, los algoritmos que se le apliquen, sobre todo los usados en ML, van a poder funcionar mejor, al operar sobre datos de menor complejidad.

7- ¿Cuantos centroides elegiste para la división del set de datos? Justifica.

  Se elijieron 578 centros, ya que lo que se buscaba es que el programa predijiera el Publisher de cada juego, por lo que al haber 578 Publisher el objetivo seria encontrar 578 Clusters, 1 por cada Publisher.

8- ¿Que observaste al graficar los puntos luego de reducir su dimensionalidad? ¿Servirá hacer clustering para este set de datos?

  Se observa que una gran parte de los datos estan dispersos de manera casi aleatortia, mientras que otros siguen una especie de linea continua. Esto nos mustra que el clustering no es ideal, por los datos dispersos. Ademas los datos mas continuos no pertenecen a un mismo Publisher, por lo que un si un cluster los clasifica juntos, por su cercania, se estaria equivocando.